<a href="https://colab.research.google.com/github/williamedwardhahn/OISC/blob/main/Hahn_MPCR_OISC_April_10_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MPCR OISC

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Thing <- Name of the thing <- Name of the Name of the thing

In [ ]:
# 0 Location of IP
# M[0] Value of IP
# i = M[0] - 2 # Location of Location of Value to be copied 
# j = M[0] - 1 # Location of Location of Value to be pasted 
# M[i]          # Location of Value to be copied.
# M[M[i]]       # Value to be copied

In [ ]:
# a  Location of Location of Value to be Copied
# b  Location of Location of Value to be Pasted
# i  Location of Value to be Copied
# j  Location of Value to be Pasted
# x  Value to be Copied
# y  Value to be Pasted

In [ ]:
%%writefile code.ai
IP
A
B
C
Add
Sub
Mult
Div
P
L
S
W
Zero 0,L L,S W,0
One 1,L L,S W,0
Halt 0,L L,0
Push A,S W,0
Peek S,A A,S W,0
Pop S,A W,0
Drop S,C W,0
Dup S,A A,S A,S W,0
Swap S,B S,A B,S A,S W,0
+ S,A S,B Add,S W,0
- S,A,S B,Sub,S W,0
* S,A S,B Mult,S W,0
/ S,A S,B Div,S W,0
Not S,A -1,L L,B Mult,A 1,L L,B Add,S W,0
Negate S,A -1,L L,B Mult,S W,0
Rot S,C S,B S,A C,S A,S B,S W,0
Mod S,A A,C S,B B,S A,S /,W B,S *,W C,S -,W W,0
Continue A,A W,0
Branch Dup,W Not,W Rot,W *,W Rot,W *,W +,W S,A A,W W,0
Square Dup,W *,W W,0
Cube Dup,W *,W W,0
Fourth Square,W Square,W A,A W,0
Double Dup,W +,W W,0
SL 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0   
WL 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
PS 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

Overwriting code.ai


In [ ]:
N = 1024

def load(filename):
    words = np.zeros(N, dtype='object')
    codes = np.zeros(N, dtype='object')
    i = 0
    with open(filename, 'r') as f:
        for line in f:
            line = line.strip().replace(',', ' ').split()
            words[i] = line[0]
            c = line[1:]
            l = len(c)
            if l > 1:
                codes[i] = i+1
                i+=1
            codes[i:i+l] = c
            i += 1
            if l > 1:
                i+= l-1
    return words,codes

In [ ]:
def D(word):  #Dictionary
    if word in words:
        return np.where(word == words)[0][0]
    else:
        return -1

In [ ]:
def recode(codes): 
    for i in range(len(codes)):
        if D(codes[i]) != -1:
            codes[i] = D(codes[i])   
    return codes

In [ ]:
def setup(program):

    program = compile_program(program)

    M = np.zeros(N, dtype=int)
    M[:len(codes)] = codes
    M[D("PS"):D("PS") + len(program)] = program
    M[D("IP")] = D("PS")
    M[D("SL")] = 0 
    M[D("WL")] = 0 
    
    return M

In [ ]:
def compile_program(X):
    program = []
    for x in X.split(' '):
        
        if x.lstrip("-").isdigit():
            program += [int(x), D('L'), D('L'), D('S')]
        elif "," in x:
            a,b = recode(x.split(','))
            program += [int(a),int(b)]
        else:
            program += [D(x), D('W')]

    program += [0, D('L'), D('L'), D('IP')]
    return program

In [ ]:
# a  Location of Location of Value to be Copied
# b  Location of Location of Value to be Pasted
# i  Location of Value to be Copied
# j  Location of Value to be Pasted
# x  Value to be Copied
# a,b = M[IP]-2,M[IP]-1 
# i,j = M[a]   ,M[b]
# i,j = M[M[IP]-2]   ,M[M[IP]-1]
# x   = M[M[i]]

In [ ]:
def run(M):

    IP,A,B,C,Add,Sub,Mult,Div,L,S,SL,W,WL = 0,1,2,3,4,5,6,7,9,10,231,11,257  #(D(x) for x in ["IP","A","B","C","Add","Sub","Mult","Div","L", "S", "SL", "W", "WL"])

    while M[IP] > 0:

        M[IP] += 2

        a,b = M[IP]-2 , M[IP]-1
        i,j = M[a]    , M[b]
        
        if i == S and j == L:
            M[L] = i
        elif i == W and j == L:
            M[L] = i

        elif i == S:
            M[j] = M[SL + M[SL]]
            M[SL] -= 1
        elif j == S:
            M[SL] += 1
            M[SL + M[SL]] = M[i]

        elif i == W:
            M[j] = M[WL + M[WL]]
            M[WL] -= 1
        elif j == W:
            M[WL] += 1
            M[WL + M[WL]] = M[0]
            M[0] = M[i]

        elif j == L:
            M[L] = i

        else:
            M[j] = M[i]

        M[Add]  = M[A] + M[B]
        M[Sub]  = M[A] - M[B]
        M[Mult] = M[A] * M[B]
        M[Div]  = 0 if M[B] == 0 else M[A] // M[B]

    return M

In [ ]:
words,codes = load('code.ai')
codes = recode(codes).astype(int)

In [ ]:
print(words.tolist())

['IP', 'A', 'B', 'C', 'Add', 'Sub', 'Mult', 'Div', 'P', 'L', 'S', 'W', 'Zero', 0, 0, 0, 0, 0, 0, 'One', 0, 0, 0, 0, 0, 0, 'Halt', 0, 0, 0, 0, 'Push', 0, 0, 0, 0, 'Peek', 0, 0, 0, 0, 0, 0, 'Pop', 0, 0, 0, 0, 'Drop', 0, 0, 0, 0, 'Dup', 0, 0, 0, 0, 0, 0, 0, 0, 'Swap', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, '+', 0, 0, 0, 0, 0, 0, 0, 0, '-', 0, 0, 0, 0, 0, 0, 0, 0, '*', 0, 0, 0, 0, 0, 0, 0, 0, '/', 0, 0, 0, 0, 0, 0, 0, 0, 'Not', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'Negate', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'Rot', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'Mod', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'Continue', 0, 0, 0, 0, 'Branch', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'Square', 0, 0, 0, 0, 0, 0, 'Cube', 0, 0, 0, 0, 0, 0, 'Fourth', 0, 0, 0, 0, 0, 0, 0, 0, 'Double', 0, 0, 0, 0, 0, 0, 'SL', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'WL', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
print(codes.tolist())

[13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 0, 9, 9, 10, 11, 0, 20, 1, 9, 9, 10, 11, 0, 27, 0, 9, 9, 0, 32, 1, 10, 11, 0, 37, 10, 1, 1, 10, 11, 0, 44, 10, 1, 11, 0, 49, 10, 3, 11, 0, 54, 10, 1, 1, 10, 1, 10, 11, 0, 63, 10, 2, 10, 1, 2, 10, 1, 10, 11, 0, 74, 10, 1, 10, 2, 4, 10, 11, 0, 83, 10, 1, 10, 2, 5, 10, 11, 0, 92, 10, 1, 10, 2, 6, 10, 11, 0, 101, 10, 1, 10, 2, 7, 10, 11, 0, 110, 10, 1, -1, 9, 9, 2, 6, 1, 1, 9, 9, 2, 4, 10, 11, 0, 127, 10, 1, -1, 9, 9, 2, 6, 10, 11, 0, 138, 10, 3, 10, 2, 10, 1, 3, 10, 1, 10, 2, 10, 11, 0, 153, 10, 1, 1, 3, 10, 2, 2, 10, 1, 10, 100, 11, 2, 10, 91, 11, 3, 10, 82, 11, 11, 0, 176, 1, 1, 11, 0, 181, 53, 11, 109, 11, 137, 11, 91, 11, 137, 11, 91, 11, 73, 11, 10, 1, 1, 11, 11, 0, 202, 53, 11, 91, 11, 11, 0, 209, 53, 11, 91, 11, 11, 0, 216, 201, 11, 201, 11, 1, 1, 11, 0, 225, 53, 11, 73, 11, 11, 0, 232, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 258, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
program = "8 9 * 7 + Fourth"

M = setup(program)
M = run(M)
print(M[D('SL') + M[D('SL')]])

38950081


In [ ]:
program = "5 Fourth"

M = setup(program)
M = run(M)
print(M[D('SL') + M[D('SL')]])

625


In [ ]:
 program = "1 2 3 Rot"

M = setup(program)
M = run(M)
print(M[D('SL') + M[D('SL')]])
print(M[D('SL'):D('SL')+10])

0
[0 0 3 3 1 2 0 0 0 0]


In [ ]:
program = "1 Not"

M = setup(program)
M = run(M)
print(M[D('SL') + M[D('SL')]])

0


In [ ]:
program = "0 Not"

M = setup(program)
M = run(M)
print(M[D('SL') + M[D('SL')]])

1


In [ ]:
program = "-5 Negate"

M = setup(program)
M = run(M)
print(M[D('SL') + M[D('SL')]])

5


In [ ]:
program = "10 11 +"
M = setup(program)
M = run(M)
print(M[D('SL'):D('SL')+10])

[ 1 21 11  0  0  0  0  0  0  0]


In [ ]:
program = "10 3 Swap"
M = setup(program)
M = run(M)
print(M[D('SL'):D('SL')+10])

[ 2  3 10  0  0  0  0  0  0  0]


In [ ]:
# Word_F Word_T b Branch
# Branch expects two word address and a bool on the stack

In [ ]:
program = "5 Double,S Fourth,S One Branch 22 1 +"

M = setup(program)
M = run(M)
print(M[D('SL'):D('SL')+10])
print(M[D('SL') + M[D('SL')]])

[  2 625  23   1   0   1   0   0   0   0]
23


In [ ]:
program = "5 Double,S Fourth,S Zero Branch 22 1 +"

M = setup(program)
M = run(M)
print(M[D('SL'):D('SL')+10])
print(M[D('SL') + M[D('SL')]])

[ 2 10 23  1  1  0  0  0  0  0]
23


In [ ]:
program = "5 Continue,S Fourth,S One Branch 22 1 +"

M = setup(program)
M = run(M)
print(M[D('SL'):D('SL')+10])
print(M[D('SL') + M[D('SL')]])

[  2 625  23   1   0   1   0   0   0   0]
23


In [ ]:
program = "3 10 Mod"
M = setup(program)
M = run(M)
print(M[D('SL'):D('SL')+10])
print(M[D('SL') + M[D('SL')]])

[ 1  1 10  0  0  0  0  0  0  0]
1


In [ ]:
program = "4 7 Mod"
M = setup(program)
M = run(M)
print(M[D('SL'):D('SL')+10])
print(M[D('SL') + M[D('SL')]])

[1 3 7 0 0 0 0 0 0 0]
3


In [ ]:
def REPL(M):

    while True:

        program = input()

        if program == "Exit": break
    
        program = compile_program(program)

        M[D("PS"):D("PS") + len(program)] = program
        M[0] = D("PS")
        
        M = run(M)

        # print(M[D('SL'):D('SL')+8])
        print(M[D('SL') + M[D('SL')]])
        # print(M[D('SL')+1:D('SL')+1+M[D('SL')]])

In [ ]:
# 8 9 * 7 + Fourth

In [ ]:
program = ""
M = setup(program)
REPL(M)

In [ ]:
max

# Notes

We are looking at a One Instruction Set Computer OISC. The single native instruction in the 'run' virtual machine is copy,paste. The rest of the operations are memory mapped. L is for literals and allows one to convert from an address to an actual value. There are two stacks, one for parameters S at location SL and a second stack for return address called W at location WL. The language in code.ai is a forth like stack language, like forth each word is a subroutine that is called with the machine code # W where # is a place holder for the word that is looked up in the dictionary D to get location in memory where the subroutine machine code is located. The compiler build a threaded interpreted language style program that is loaded on the memory tape M and then run. This is an example of subroutine threading. Copying to W is a call (JSR) and copying from W to 0 is a return from subroutine (RTS).